In [32]:
#importing Libraries
import numpy as np
import pandas as pd
from scipy.optimize import root_scalar
from scipy import interpolate
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
pio.renderers.default = 'iframe'

In [33]:
#Functions for curvature and graphing values
x = np.linspace(0, 1, 200)
alpha = (x/2 - 0.3)**2 + 0.05 
beta = -alpha

#Variables and Constants 
p_0i = 140 # kPa
T_0i = 290 # K
R = 287 # J/kgK
c_p = 1006 # J/kgK
g = 1.4
shock_x = 0.8 
throat_x = 0.6

#Assuming flow is chocked, Mach number can be defined as 1
M_t = 1

In [34]:
#Function for area
def area(x):
    area = 2 * ((x/2 - 0.3)**2 + 0.05)
    return area

In [35]:
#Finding the area at the throat, solving for Mach Number
A_t = area(throat_x)

# Define a mach function to calculate the roots of the area ratio mach relation set to zero
def mach_function(M, x_pos):
    A = area(x_pos)
    return np.sqrt((1/M**2)*((2/(g+1))*(1+((g-1)/2)*M**2))**((g+1)/(g-1))) - A/A_t

In [36]:
#Find Mach number at shock position, the minimum of the mach value at the entrance, and the mach number after the shock
M_s_root = root_scalar(mach_function, args = (shock_x), x0 = 1, x1 = 2.5)
M_s = M_s_root.root

M_min_root = root_scalar(mach_function, args = (0), x0 = 0.2, x1 = 1)
M_min = M_min_root.root

M_as = np.sqrt(((g-1)*M_s**2 + 2)/(2*g*M_s**(2)-(g-1)))

In [37]:
#Functions to calculate stagnation pressure before and after shock
def stag_pressure(x_pos):
    if x_pos < shock_x:
        stag_pressure = p_0i       
    else:
        stag_pressure = p_0i * ((((g + 1) * M_s**2) / ((g - 1) * M_s**2 +2))**(g/(g - 1))) * ((g + 1) / (2*g* M_s**2 - (g -1)))**(1/(g-1))
    return stag_pressure

#Mass flow rate at the throat (M=1)
m_dot = ((A_t)*(p_0i*1000)/(np.sqrt(T_0i)))*np.sqrt(g/R)*((g+1)/2)**(-(g+1)/(2*(g-1))) 

#Function for the roots of the mass flow rate equation set to zero
def mach_function_as(M, x_pos):
    A = area(x_pos)
    p_0 = stag_pressure(x_pos)*1000
    return (A*p_0/np.sqrt(T_0i))*(np.sqrt(g/R))*M*(1+((g-1)/2)*M**2)**(-(g+1)/(2*(g-1))) - m_dot

In [38]:
#Function for calculateing Mach number at an x value

def mach_number(x_pos):
    if x_pos <= 0.5:
        M_root = root_scalar(mach_function, args = (x_pos), x0 = M_min, x1 = 0.8)
        M = M_root.root
    elif 0.5 < x_pos <= throat_x: 
        M_root = root_scalar(mach_function, args = (x_pos), x0 = M_min, x1 = 1)
        M = M_root.root
    elif x_pos < shock_x:
        M_root = root_scalar(mach_function, args = (x_pos), x0 = 1, x1 = M_s)
        M = M_root.root
    else: 
        M_root = root_scalar(mach_function_as, args = (x_pos), x0 = M_as, x1 = 0.2)
        M = M_root.root
    return M

In [39]:
#Function for calculating static pressure
def static_pressure(x_pos):
    M = mach_number(x_pos)
    p_0 = stag_pressure(x_pos)
    static_pressure = p_0 * (1 + ((g - 1)/2)*M**2)**(-g/(g-1))
    return static_pressure

In [40]:
#Function for calculating static temperature
def static_temperature(x_pos):
    M = mach_number(x_pos)
    static_temperature = T_0i * (1 + ((g - 1)/2)*M**2)**(-1)
    return static_temperature

In [41]:
#Function for calculating the speed of the particle based of position
def speed(x_pos):
    M = mach_number(x_pos)
    T = static_temperature(x_pos)
    a = np.sqrt(g*R*T)
    v = M * a
    return v

In [42]:
#Using functions above, can find following fields of interest from the x value
M_values = []
p0_values = []
p_values = []
T_values = []
v_values = []
for x_val in x:
    M_values.append(mach_number(x_val))
    p0_values.append(stag_pressure(x_val))
    p_values.append(static_pressure(x_val))
    T_values.append(static_temperature(x_val))
    v_values.append(speed(x_val))

In [43]:
Can create a plot
fig = go.Figure()

fig.add_scatter(x=x, y=M_values, line_color = 'plum')
fig.add_scatter(x=x, y=p0_values, line_color = 'mediumaquamarine', visible=False)
fig.add_scatter(x=x, y=p_values, line_color = 'cornflowerblue', visible=False)
fig.add_scatter(x=x, y=T_values, line_color = 'tomato', visible=False)
fig.add_scatter(x=x, y=v_values, line_color = 'darkmagenta', visible=False)

fig.update_layout(updatemenus = [dict(active=0, buttons = [dict(label='Mach Number', method='update', args=[{'visible':
                  [True, False, False, False, False]}, {'yaxis':{'title':'Mach Number'}}]), dict(label='Stagnation Pressure',
                  method='update', args=[{'visible': [False, True, False, False, False]}, {'yaxis':{'title':
                  'Stagnation Pressure (kPa)'}}]), dict(label='Static Pressure', method='update', args=[{'visible': [False, 
                  False, True, False, False]}, {'yaxis':{'title':'Static Pressure (kPa)'}}]), dict(label='Static Temperature'
                  , method='update', args=[{'visible': [False, False, False, True, False]}, {'yaxis':{'title':
                  'Static Temperature (K)'}}]), dict(label='Velocity', method='update', args=[{'visible':[False, False, 
                  False, False, True]}, {'yaxis':{'title':'Velocity (m/s)'}}])])])

fig.update_yaxes(title_text='Mach Number')
fig.update_xaxes(title_text="x position")
fig.show()

In [44]:
#Can create contour map from nozzle
fig = go.Figure()

fig.add_contour(z=[M_values, M_values], x=x, contours_coloring = 'heatmap', line_width = 0, colorscale = 'deep',
                reversescale = True, colorbar=dict(title='Mach Number', titleside='right'))
fig.add_contour(z=[p0_values, p0_values], x=x, contours_coloring ='heatmap', line_width = 0, colorscale = 'Teal', 
                visible = False, colorbar=dict(title='Stagnation Pressure (kPa)', titleside='right'))
fig.add_contour(z=[p_values, p_values], x=x, contours_coloring = 'heatmap', line_width = 0, colorscale = 'viridis', 
                visible = False, colorbar=dict(title='Static Pressure (kPa)', titleside='right'))
fig.add_contour(z=[T_values, T_values], x=x, contours_coloring = 'heatmap', line_width = 0, colorscale = 'blackbody', 
                reversescale = True, visible = False, colorbar=dict(title='Static Temperature (K)', titleside='right'))

fig.add_scatter(x=x, y=alpha+0.15, line_color = 'white')
fig.add_scatter(x=x, y=x-x+0.3, fill='tonexty', fillcolor='white', line_color='white')
fig.add_scatter(x=x, y=beta+0.15, fill='tozeroy', fillcolor='white', line_color='white')

fig.update_layout(showlegend=False, updatemenus=[dict(active=0, buttons = [dict(label='Mach Number', method='update', args=
                  [{'visible':[True, False, False, False, True, True, True]}]), dict(label='Stagnation Pressure', 
                  method='update', args=[{'visible':[False, True, False, False, True, True,True]}]), dict(label=
                  'Static Pressure', method='update', args=[{'visible':[False, False, True, False, True, True, True]}]),
                  dict(label='Static Temperature', method='update', args=[{'visible':[False, False, False, True, True, True,
                  True]}])], y=1.2, x=1.1)])

fig.update_yaxes(range=[0, 0.3], showticklabels = False)
fig.update_xaxes(title='x position')

fig.show()


In [45]:
#Interval of time between each x position
dx = np.diff(x)
vav = []
for i in range(0, len(dx)):
    av = sum(v_values[i: i+1]) / 2
    vav.append(av)
dt = dx / vav

# find the time
time = []
for j in range(0, len(x)):
    time.append(sum(dt[0:j]))
    
# interpolate using the x and time values then use the interpolation function to find x positions at equal intervals of time
f = interpolate.interp1d(x=time, y=x)
tnew = np.linspace(0, max(time), 500)
xnew = f(tnew)

In [46]:
# using the interpolated data we can animate how a particle would move through the nozzle
fig = go.Figure()

fig.add_scatter(x=x, y=np.zeros(500))
fig.add_scatter(x=x, y=np.zeros(500), line_dash = 'dot', line_color = 'lightsteelblue')

fig.add_scatter(x=x, y=alpha, line_color = 'black')
fig.add_scatter(x=x, y=beta, line_color = 'black')
fig.add_vline(x = shock_x, line_dash = 'dash', line_color = 'red', annotation_text = 'shock')
fig.add_vline(x = throat_x, line_dash = 'dash', line_color = 'blue', annotation_text = 'throat')

fig.update_layout(xaxis=dict(range=[0,1], autorange=False), yaxis=dict(range=[-0.15, 0.15], autorange=False), updatemenus=
                  [dict(type='buttons', buttons=[dict(label='play', method='animate', args=[None, {'frame':{'duration':10}}])])])
fig.update(frames = [go.Frame(data=[go.Scatter(x=[xnew[k]], y=[np.zeros(500)[k]], mode='markers', marker=dict(color='teal',
                    size=10))]) for k in range(500)], layout_showlegend=False)

fig.show()

In [47]:
# define x and v values for 18 different positions to create arrows
x_18 = np.linspace(0,1,18)
v_18 = []
for x_val in x_18:
    v_18.append(speed(x_val) / 1000)
    
alpha_2 = (x_18/3 -0.2)**2 + 0.025
beta_2 = -alpha_2
slope_2 = 2*(x_18/3 - 0.2)/3 * abs(np.append(np.sin(np.arctan(np.diff(alpha_2) / np.diff(x_18))), 0))

In [48]:
# create velocity vector field 
fig = ff.create_quiver(x=np.linspace(0,1,18), y=np.zeros(18), u=v_18, v=np.zeros(18), line_color = 'blue')
fig2 = ff.create_quiver(x=x_18, y=alpha_2, u=v_18, v=slope_2, line_color = 'blue')
fig3 = ff.create_quiver(x=x_18, y=beta_2, u=v_18, v=-slope_2, line_color = 'blue')

fig.add_scatter(x=x, y=alpha, line_color = 'black')
fig.add_scatter(x=x, y=beta, line_color='black')
fig.update_layout(xaxis=dict(range=[0,1]), yaxis=dict(range=[-0.15, 0.15]), showlegend = False)

fig.add_traces(data = fig2.data)
fig.add_traces(data = fig3.data)
fig.show()

